# **Data source**
https://data.binance.vision/?prefix=data/spot/daily/aggTrades/
* Aggregate tradeID
* Price
* Quantity
* First tradeId
* Last tradeID
* Timestamp
* Was the buyer the maker?
* Was the trade the best price match?

In [120]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time


import plotly.express as px

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10,5)

# **Create function to pull multiple days of tick data**

In [95]:
def get_binance_tick(symbol='ETHUSDT', aest=datetime.now().date()):
  base_url = 'https://data.binance.vision/data/spot/daily/aggTrades/'
  symbol_params = symbol + '/' + symbol
  path = '-aggTrades-'
  date_params = str(aest)
  file_type = '.zip'

  url = base_url + symbol_params + path + date_params + file_type

  df = pd.read_csv(url).iloc[:,[1,2,5,6]]
  df.columns = ['Price', 'Quantity', 'Timestamp', 'Was the buyer the maker?']
  df.Timestamp = pd.to_datetime(df.Timestamp, unit='ms')
  return df

# **Select date range**

In [168]:
# YYYY-MM-DD
t0 = '2022-1-1'
t1 = '2022-7-1'

list_days = [i.date() for i in pd.date_range(start=t0, end=t1, freq='D')]

# **Pull Data**

In [169]:
crypto_symbol = 'ETHUSDT'

df_list = [get_binance_tick(symbol='ETHUSDT', aest = list_days[i]) for i in range(0,len(list_days))]

df = pd.concat(df_list)

In [170]:
df

,Price,Quantity,Timestamp,Was the buyer the maker?
0,1941.90,0.0275,2022-06-01 00:00:00.003,True
1,1941.90,0.0147,2022-06-01 00:00:00.003,True
2,1941.91,0.0363,2022-06-01 00:00:00.008,False
3,1941.90,0.0393,2022-06-01 00:00:00.009,True
4,1941.90,0.0053,2022-06-01 00:00:00.014,True
...,...,...,...,...
823966,1059.72,0.0558,2022-07-01 23:59:59.994,True
823967,1059.72,0.0753,2022-07-01 23:59:59.994,True
823968,1059.72,0.0920,2022-07-01 23:59:59.994,True
823969,1059.73,0.0222,2022-07-01 23:59:59.996,False


# Agg Daily Price Volume Data

In [171]:
df1 = df.copy()

df1.Timestamp = df1.Timestamp.dt.date
df1['maker'] = np.where(df1['Was the buyer the maker?'] == True, df1['Quantity'], 0)
df1.drop('Was the buyer the maker?', axis=1, inplace=True)
df1.Price = round(df1.Price/.05) * .05
df1 = df1.groupby(['Timestamp', 'Price']).agg({"Quantity": 'sum', "maker": 'sum'}).reset_index()

df1

,Timestamp,Price,Quantity,maker
0,2022-06-01,1763.75,2.2150,1.3430
1,2022-06-01,1763.80,0.2009,0.2009
2,2022-06-01,1763.90,0.6252,0.5252
3,2022-06-01,1763.95,0.0119,0.0119
4,2022-06-01,1764.00,9.0877,1.3057
...,...,...,...,...
77813,2022-07-01,1116.80,0.7697,0.0000
77814,2022-07-01,1116.85,6.7981,0.0000
77815,2022-07-01,1116.90,14.9590,0.0000
77816,2022-07-01,1116.95,1.5123,0.0000


# Connect to PostgreSQL

In [175]:
from sqlalchemy import create_engine
import psycopg2

In [176]:
conn = psycopg2.connect('host=localhost user=postgres password=password')

# Create Cursor object and set auto commit

In [177]:
cur = conn.cursor()
conn.set_session(autocommit=True)

# Create database

In [180]:
cur.execute('create database myfirstdb')

# Close connection & Connect to new db

In [192]:
conn.close()

In [212]:
conn = psycopg2.connect('host=localhost dbname=myfirstdb user=postgres password=password')

In [213]:
cur = conn.cursor()
conn.set_session(autocommit=True)

# SQL: Create table

In [238]:
cur.execute('''
DROP TABLE IF EXISTS crypto_ticks''')

In [239]:
sql = '''CREATE TABLE IF NOT EXISTS crypto_ticks(
dt DATE, 
close DOUBLE PRECISION, 
volume DOUBLE PRECISION, 
maker DOUBLE PRECISION
)'''

In [240]:
cur.execute(sql)

# Save dataframe on disk as csv

In [198]:
aggvol.to_csv('aggvol_testset.csv', header=False, index=False)

In [199]:
pd.read_csv('aggvol_testset.csv')

,2022-01-01,3673.4500000000003,4.1265,0.31279999999999997
0,2022-01-01,3673.50,0.0075,0.0075
1,2022-01-01,3673.55,1.5891,0.0792
2,2022-01-01,3673.60,1.1033,0.9259
3,2022-01-01,3673.65,1.6462,1.6462
4,2022-01-01,3673.80,5.0008,1.0841
...,...,...,...,...
615903,2022-07-01,1116.80,0.7697,0.0000
615904,2022-07-01,1116.85,6.7981,0.0000
615905,2022-07-01,1116.90,14.9590,0.0000
615906,2022-07-01,1116.95,1.5123,0.0000


# Open csv and copy to SQL db

In [245]:
st = time.time() 
with open('aggvol_testset.csv', 'r') as f:
    cur.copy_from(f, 'crypto_ticks', sep=",")

et = time.time()    
print('Data inserted using copy_from() successfully...')
print(str(round(et-st,6)))

Data inserted using copy_from() successfully...
2.689879


# SQL: Query the database

In [246]:
sql = '''
SELECT * FROM crypto_ticks'''

In [247]:
cur.execute(sql)

In [248]:
sql_df = pd.DataFrame(cur.fetchall(), columns = ['timestamp', 'price', 'volume', 'maker'])

In [249]:
sql_df

,timestamp,price,volume,maker
0,2022-01-01,3673.45,4.1265,0.3128
1,2022-01-01,3673.50,0.0075,0.0075
2,2022-01-01,3673.55,1.5891,0.0792
3,2022-01-01,3673.60,1.1033,0.9259
4,2022-01-01,3673.65,1.6462,1.6462
...,...,...,...,...
615904,2022-07-01,1116.80,0.7697,0.0000
615905,2022-07-01,1116.85,6.7981,0.0000
615906,2022-07-01,1116.90,14.9590,0.0000
615907,2022-07-01,1116.95,1.5123,0.0000
